In [ ]:
import pandas as pd
import numpy as np
import random

In [ ]:
n_splits=10
target_col = "HeartDiseaseorAttack"
missing_sample_size = 0.05

In [ ]:
# Read folds that are available
# Create empty lists to store train and test DataFrames
train_datasets = []
test_datasets = []

for fold_num in range(1, n_splits+1):
    train_file_path = f"folds/fold_{fold_num}_train.csv"
    test_file_path = f"folds/fold_{fold_num}_test.csv"
    
    # Load the train and test fold data into DataFrames
    train_fold = pd.read_csv(train_file_path)
    test_fold = pd.read_csv(test_file_path)
    
    train_datasets.append(train_fold)
    test_datasets.append(test_fold)
print("Folds data were loaded successfully!")

In [ ]:
for fold_num, (train_data, test_data) in enumerate(zip(train_datasets, test_datasets),1):

    if fold_num == 1:
        patients_to_nan = train_data.sample(frac=missing_sample_size, random_state=42, ignore_index=True)
        features = list(train_data.columns)
        features.remove(target_col)

patients_to_nan.to_csv(f'folds/naned/patients_to_nan_train.csv', index=False)


patient_nan_features_dict = {}
for patient_idx, patient_row in patients_to_nan.iterrows():
    # select how many features and which ones will be NaNed out for a patient idx
    sel_columns = random.choice([1,2])
    columns_to_add_nans = random.sample(features, sel_columns)
    patient_nan_features_dict[patient_idx] = columns_to_add_nans    

In [ ]:
patient_nan_dict = patients_to_nan.to_dict(orient='index')

In [ ]:
# for every fold find that patient in train data and if exists - NaN out some features
for fold_num, (train_data, test_data) in enumerate(zip(train_datasets, test_datasets),1):

    for ft_idx, ft_row in train_data.iterrows():

        for patient_idx, patient_row in patient_nan_dict.items():

            if patient_row == ft_row.to_dict():
                # print(f'Found a patient in train set in fold {fold_num} -> imputing NaN in {columns_to_add_nans} columns')        
                for col in patient_nan_features_dict[patient_idx]:
                    train_data.at[ft_idx, col] = np.NaN
            # patients are unique (dataframe has no duplicates) - so once one patient is found in one folds' training set, the search in that fold can be finished
            # break
    print(f'Fold {fold_num} finished successfully')
    # Save each fold as a CSV file
    train_data.to_csv(f'folds/naned/fold_{fold_num}_train.csv', index=False)
    test_data.to_csv(f'folds/naned/fold_{fold_num}_test.csv', index=False)

